# **ERP and time-frequency analysis in Python and MNE**

### **Working with trial-based data in Python**

In [ ]:
#Imporging necessary libraries
from scipy.io import loadmat
#Loading MATLAB data
data = loadmat('Trial-based.mat')
type(data)
data.keys()

In [ ]:
#View the data associated with keys
for key in data.keys():
    print(key, data[key])


In [ ]:
#Exctract conditions of the experiment
conditionHappyFace = data['EEGa']
conditionNonFace = data['EEGb']

#Extract time
time = data['t'][0]
%whos


In [ ]:
#Define number of trials
trialNumber = conditionHappyFace.shape[0]

#Define number of samples in each trial
sampleNumber = conditionHappyFace.shape[1]


In [ ]:
conditionHappyFace.shape[0]

In [ ]:
#Print first/last trial
print(conditionHappyFace[1000])

### **Plotting single trials**

In [ ]:
#Ploting one trial for each condition of the experiment
import matplotlib.pyplot as plt
plt.figure(figsize=(12,1))
plt.plot(time, conditionHappyFace[999], linewidth=2)
plt.plot(time, conditionNonFace[999])



In [ ]:
plt.figure(figsize=(12,1))
plt.plot(time*1000, conditionHappyFace[0], label='Happy Face')
plt.plot(time*1000, conditionNonFace[0], label='Non-Face')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage [$\mu$ V]')
plt.title('EEG data from both conditions, Trial 1')

plt.plot([250,250],[-5,5], 'r', lw=2, label='stimulus', alpha=.8, ls='--')

plt.legend()



### **Computing ERP for the conditions**

In [ ]:
#Compute mean ERPs over the first (raw) dimention
ERPHappy = conditionHappyFace.mean(0)
ERPNonFace = conditionNonFace.mean(0)

### **Plotting mean ERPs**

In [ ]:
plt.figure(figsize=(12,2))

plt.plot(time*1000, ERPHappy, label='ERP for Happy Face Condition')
plt.plot(time*1000, ERPNonFace, label='ERP for Non Face Condition')
plt.xlabel('Time (s)')
plt.ylabel('Voltage [$\mu$ V]')
plt.title('Mean ERPs for two conditions')

plt.plot([250, 250], [-.3,.3], 'r', lw=2, label='Stimulus', alpha=0.8, ls='--')
plt.legend()


### **ERP analysis in MNE**

### **Importing EEG in MNE**

In [ ]:
#importing necessary libraries
import mne
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

#Importing EDF file
EEG = mne.io.read_raw_edf('ERD_ERS.edf', preload=True)


In [ ]:
#Examine the EEG channels
EEG.ch_names

In [ ]:
#Remove desired channel
EEG.drop_channels('LABEL')

In [ ]:
EEG.ch_names

### **Montage set**

In [ ]:
montage = mne.channels.make_standard_montage('standard_1020')
EEG.set_montage(montage, on_missing='ignore')


In [ ]:
#Filter the data
EEG.filter(0.1, 30)

In [ ]:
EEG.plot(block=False, duration=10.0)

### **Loading the Labels file and creating Events**

In [ ]:
# Loading trigger codes from Labels.txt
labelsData = np.loadtxt('Labels.txt', dtype=float)
triggerCodes = labelsData[:, 0]
# Extracting sampling frequency from EEG info
sfreq = EEG.info['sfreq']

# Converting trigger codes to trigger samples
triggerSamples = (triggerCodes * sfreq ).astype(int)

# Extracting condition labels
conditionLabels = (labelsData[:,1]).astype(int)

# Creating EEG events array with trigger samples and condition labels
EEGEvents = np.column_stack((triggerSamples, np.zeros_like(triggerSamples), conditionLabels))
conditionLabels

### **Epoching the events for MNE**

In [ ]:
#Plotting and seeeing events
EEG.plot(block=False, duration=10.0, events = EEGEvents, title = 'EEG with Events')

In [ ]:
#Adding preferred labels for the events
eventIDs = {'Condition 1':1, 'Condition 2':2, 'Condition 3':3, 'Condition 4':4 }

EEG.plot(duration=10.0, events = EEGEvents, title='EEG with Events', event_id = eventIDs)

In [ ]:
#Creating a variable for EVENTS structure
epochStart= -.2
epochEnd=2
conditionEpochs = mne.Epochs (EEG, events= EEGEvents, event_id=eventIDs, tmin=epochStart, tmax=epochEnd,
            baseline=(-0.2, 0), reject=None)
conditionEpochs

In [ ]:
#Access specific epochs
conditionEpochs['Condition 1']

### **Move epochs to different variables**

In [ ]:
Condition1Epochs = conditionEpochs['Condition 1']
Condition2Epochs = conditionEpochs['Condition 2']
Condition1Epochs.get_data()[0,8,:]

### **Compute mean ERPs**

In [ ]:
averageERPCondition1 = Condition1Epochs.average()
averageERPCondition2 = Condition2Epochs.average()
averageERPCondition1

### **Pick a channel**

In [ ]:
channelForAnalysis = 'Fz-Av'
myPick = EEG.info['ch_names']
myPick = EEG.info['ch_names'].index(channelForAnalysis)

In [ ]:
EEG.ch_names

### **Mean for one channel**

In [ ]:
averageERPCondition1 = Condition1Epochs.average()
averageERPCondition2 = Condition2Epochs.average()

### **Plot means for desired channels**

In [ ]:
averageERPCondition1.plot(time_unit='ms')
averageERPCondition2.plot(time_unit='ms')

### **Time-frequency in MNE**

In [ ]:
from mne.time_frequency import tfr_morlet
freqRange = np.arange(4,30,.1)
numberOfCycles = 3

TFRCondition1 = tfr_morlet(Condition1Epochs, freqRange, numberOfCycles, average=True)
TFRCondition2 = tfr_morlet(Condition2Epochs, freqRange, numberOfCycles, average=True)


### **Plotting time-frequency for our pick**

In [ ]:
TFRCondition1

In [ ]:
TFRCondition1_Fz = TFRCondition1[0].copy()
TFRCondition1_Fz.plot(myPick, baseline=(-0.2, 0), mode='logratio',
                      title='Time-Frequency for Condition 1 (Fz)')